# Parsing APIs Example

## Intro

Now we will take a look on a real data. When you parse data from web you will often meet API based web-pages. 

For example [zalando.fr](https://www.zalando.fr/accueil-homme/) is API based web-page. 

In this guided lab you will learn how to obtain the links from webpages and extract the data. Read through this doc, execute the cells in order and make sure you understand the explanations. 

*Note: This guided lab uses Google Chrome. Other browsers like Safari and Firefox have similar tools for developers but they work differently. To save your time in following this lab, it is strongly recommended that you install and use Google Chrome.*

## Obtaining the link

Zalando is discount e-store where you can buy clothes and accesories with discount. When we go to the web-page, we can choose different sections. First the general process will be shown using [Children section](https://www.zalando.fr/accueil-enfant/) as example.

Here we will parse data about promotions only. Therefore, final output will be the DataFrame with all the goods under discount.

[![Image from Gyazo](https://i.gyazo.com/fa4874d8e81c7570273bbfb853d66308.png)](https://gyazo.com/fa4874d8e81c7570273bbfb853d66308)


We go to Promos page. Right click of mouse shows us a list of actions possible, from which we select Inspect.

<img src='https://i.gyazo.com/bccbd11d69c9040dc98758d443e32052.png' width="400">


You will see the menu dropdown on the right side or on the bottom of the window. There you should click on Network:


[![Image from Gyazo](https://i.gyazo.com/f7e0db81cbfee67694183d1a7640bf81.png)](https://gyazo.com/f7e0db81cbfee67694183d1a7640bf81)

Right after the developer part will change showing the files behind the page. In order to obtain only useful files we select the following settings:
1. Preserve Log
2. Select XHR files.

[![Image from Gyazo](https://i.gyazo.com/9a899d4441d9d93e795f79747f1e47d5.png)](https://gyazo.com/9a899d4441d9d93e795f79747f1e47d5)

In order to obtain some files we need to scrool down and go forward to second page. 

[![Image from Gyazo](https://i.gyazo.com/0956eb3d5125075a236c9a439c7749c7.png)](https://gyazo.com/0956eb3d5125075a236c9a439c7749c7)

In the Network panel you can see the following files being uploaded. All the data on the web-page is uploaded from the json file, which is one of the following. It is important to understand which file contains what kind of information. 

<a href="https://gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c"><img src="https://i.gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c.png" alt="Image from Gyazo" width="724.8"/></a>

When you find what kind of information you need for the data to be uploaded you just test it. Here we need the article... file:

<a href="https://gyazo.com/78b35bf492994b3f35c0564a21da202a"><img src="https://i.gyazo.com/78b35bf492994b3f35c0564a21da202a.png" alt="Image from Gyazo" width="727.2"/></a>

When we test the link in Chrome inkognito mode we obtain the proper json file:


<a href="https://gyazo.com/b60453fa98454fa29771c731a5174443"><img src="https://i.gyazo.com/b60453fa98454fa29771c731a5174443.png" alt="Image from Gyazo" width="1530.4"/></a>

In order to change the objects in the json file (kind of pagination), you need to change the offset (the number of the first element on the page). in fact, if you take a look on the link, it is easy to unerstand the structure of the link.

# Reading the data

Now the party rocks! When we know how can we obtain the data, it is not a problem to obtain the whole database with all the data from the web-page.
In this lab you will collect your database of Zalando products. You select which goods you want to track. You can define as many filters to your data as you want. Just make sure that the data represents the filters.




In [13]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize


In [14]:
# Paste the url you obtained for your data
url='https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset=84&sort=sale'


#### Collect first 84 object of the of the data (1st page)

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having *family_articles, flags, media* and *sizes* remaining lists (they are exceptions).

In [15]:
import urllib.request
uf = urllib.request.urlopen('https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset=84&sort=sale')
response = uf.read().decode('utf-8')
results = json.loads(response)
flattened_data = json_normalize(results)
flattened_data1 = json_normalize(flattened_data.articles[0])
flattened_data1


,brand_name,family_articles,flags,is_premium,media,name,price.has_different_original_prices,price.has_different_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,sizes,sku,url_key
0,Nike Sportswear,"[{'sku': 'NI114D07D-A12', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': '-20%', 'tra...",False,[{'path': 'NI/11/4D/07/DA/12/NI114D07D-A12@11....,CORTEZ BASIC - Baskets basses - white / varsi...,False,False,False,False,"64,95 €","52,00 €",shoe,"[35.5, 36, 36.5, 37.5, 38, 38.5, 39, 40]",NI114D07D-A12,nike-sportswear-cortez-basic-baskets-basses-wh...
1,Nike Sportswear,"[{'sku': 'NI114D0AI-C11', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': '-20%', 'tra...",False,[{'path': 'NI/11/4D/0A/IC/11/NI114D0AI-C11@6.j...,COURT BOROUGH - Baskets basses - atmosphere gr...,False,False,False,False,"39,95 €","32,00 €",shoe,"[28, 28.5, 29.5, 30, 31.5, 32, 33.5, 34, 35]",NI114D0AI-C11,nike-sportswear-court-borough-low-baskets-bass...
2,Joha,"[{'sku': 'JH123B00Q-A11', 'url_key': 'joha-leg...","[{'key': 'discountRate', 'value': '-30%', 'tra...",False,[{'path': 'JH/12/3B/00/QA/11/JH123B00Q-A11@6.j...,LEGGINGS FOOT BABY - Pantalon classique - offw...,False,False,False,False,"19,95 €","14,00 €",clothing,"[1-2a, 1-4m, 4-9m, 9-12m]",JH123B00Q-A11,joha-leggings-foot-baby-pantalon-classique-off...
3,Nike Sportswear,"[{'sku': 'NI114D03N-Q11', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': '-20%', 'tra...",False,[{'path': 'NI/11/4D/03/NQ/11/NI114D03N-Q11@12....,MD RUNNER 2 - Baskets basses - black/white/wol...,False,False,False,False,"39,95 €","32,00 €",shoe,"[28, 28.5, 29.5, 30, 31.5, 32, 33.5, 34, 35]",NI114D03N-Q11,nike-sportswear-md-runner-2-baskets-basses-bla...
4,adidas Originals,"[{'sku': 'AD116D009-Q12', 'url_key': 'adidas-o...","[{'key': 'discountRate', 'value': 'Jusqu’à -30...",False,[{'path': 'AD/11/6D/00/9Q/12/AD116D009-Q12@12....,SUPERSTAR FOUNDATION - Baskets basses - core b...,False,True,True,False,"69,95 €","49,00 €",shoe,"[36, 38, 35 1/2, 36 2/3, 37 1/3, 38 2/3]",AD116D009-Q12,adidas-originals-superstar-foundation-baskets-...
5,New Look 915 Generation,"[{'sku': 'NL623D01Q-K11', 'url_key': 'new-look...","[{'key': 'discountRate', 'value': '-20%', 'tra...",False,[{'path': 'NL/62/3D/01/QK/11/NL623D01Q-K11@6.j...,PLAIN BARDOT PLAYSUIT - Combinaison - mid blue,False,False,False,False,"14,95 €","12,00 €",clothing,"[10a, 11a, 12a, 14a, 15a]",NL623D01Q-K11,new-look-915-generation-plain-bardot-playsuit-...
6,Geox,"[{'sku': 'GE114G01K-K11', 'url_key': 'geox-kay...","[{'key': 'discountRate', 'value': '-30%', 'tra...",False,[{'path': 'GE/11/4G/01/KK/11/GE114G01K-K11@3.j...,KAYTAN BOY - Chaussures premiers pas - navy/avio,False,False,False,False,"48,95 €","34,27 €",shoe,"[20, 21, 22, 23, 24, 25]",GE114G01K-K11,geox-kaytan-boy-sandales-navyavio-ge114g01k-k11
7,Jordan,"[{'sku': 'JOC16D00B-Q11', 'url_key': 'jordan-l...","[{'key': 'discountRate', 'value': 'Jusqu’à -40...",False,[{'path': 'JO/C1/6D/00/BQ/11/JOC16D00B-Q11@9.j...,LEGACY 312 - Baskets montantes - black/varsit...,False,True,True,False,"74,95 €","45,00 €",shoe,"[27.5, 28, 28.5, 29.5, 30, 31, 31.5, 32, 33, 3...",JOC16D00B-Q11,jordan-legacy-312-baskets-montantes-blackvarsi...
8,Timberland,"[{'sku': 'TI116I010-B11', 'url_key': 'timberla...","[{'key': 'discountRate', 'value': 'Jusqu’à -40...",False,[{'path': 'TI/11/6I/01/0B/11/TI116I010-B11@10....,PREMIUM BOOT - Bottines à lacets - wheat,False,True,True,False,"149,95 €","90,00 €",shoe,"[36, 37, 38, 39, 40]",TI116I010-B11,timberland-premium-boot-bottines-a-lacets-whea...
9,Kickers,"[{'sku': 'KI113F01Q-J11', 'url_key': 'kickers-...","[{'key': 'discountRate', 'value': '-35%', 'tra...",False,[{'path': 'KI/11/3F/01/QJ/11/KI113F01Q-J11@11....,BONBEK - Chaussures premiers pas - pink,False,True,True,False,"59,95 €","39,00 €",shoe,"[22, 23, 24, 25, 26, 27]",KI113F01Q-J11,kickers-bonbek-chaussures-premiers-pas-pink-ki...


#### Collect all the objects from selected filters. Total number of pages can be found in the same json. Use *sku* column as index.

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having family_articles, flags, media and sizes remaining lists (they are exceptions).

In [16]:
total_pages=results['pagination']['page_count']
df=pd.DataFrame()
for i in range(total_pages):
    k=84*i
    try:
        flattened_data1=flattened_data1.set_index('sku')
    except:
        continue
    df = df.append(flattened_data1)
df

,brand_name,family_articles,flags,is_premium,media,name,price.has_different_original_prices,price.has_different_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,sizes,url_key
sku,,,,,,,,,,,,,,,
NI114D07D-A12,Nike Sportswear,"[{'sku': 'NI114D07D-A12', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': '-20%', 'tra...",False,[{'path': 'NI/11/4D/07/DA/12/NI114D07D-A12@11....,CORTEZ BASIC - Baskets basses - white / varsi...,False,False,False,False,"64,95 €","52,00 €",shoe,"[35.5, 36, 36.5, 37.5, 38, 38.5, 39, 40]",nike-sportswear-cortez-basic-baskets-basses-wh...
NI114D0AI-C11,Nike Sportswear,"[{'sku': 'NI114D0AI-C11', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': '-20%', 'tra...",False,[{'path': 'NI/11/4D/0A/IC/11/NI114D0AI-C11@6.j...,COURT BOROUGH - Baskets basses - atmosphere gr...,False,False,False,False,"39,95 €","32,00 €",shoe,"[28, 28.5, 29.5, 30, 31.5, 32, 33.5, 34, 35]",nike-sportswear-court-borough-low-baskets-bass...
JH123B00Q-A11,Joha,"[{'sku': 'JH123B00Q-A11', 'url_key': 'joha-leg...","[{'key': 'discountRate', 'value': '-30%', 'tra...",False,[{'path': 'JH/12/3B/00/QA/11/JH123B00Q-A11@6.j...,LEGGINGS FOOT BABY - Pantalon classique - offw...,False,False,False,False,"19,95 €","14,00 €",clothing,"[1-2a, 1-4m, 4-9m, 9-12m]",joha-leggings-foot-baby-pantalon-classique-off...
NI114D03N-Q11,Nike Sportswear,"[{'sku': 'NI114D03N-Q11', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': '-20%', 'tra...",False,[{'path': 'NI/11/4D/03/NQ/11/NI114D03N-Q11@12....,MD RUNNER 2 - Baskets basses - black/white/wol...,False,False,False,False,"39,95 €","32,00 €",shoe,"[28, 28.5, 29.5, 30, 31.5, 32, 33.5, 34, 35]",nike-sportswear-md-runner-2-baskets-basses-bla...
AD116D009-Q12,adidas Originals,"[{'sku': 'AD116D009-Q12', 'url_key': 'adidas-o...","[{'key': 'discountRate', 'value': 'Jusqu’à -30...",False,[{'path': 'AD/11/6D/00/9Q/12/AD116D009-Q12@12....,SUPERSTAR FOUNDATION - Baskets basses - core b...,False,True,True,False,"69,95 €","49,00 €",shoe,"[36, 38, 35 1/2, 36 2/3, 37 1/3, 38 2/3]",adidas-originals-superstar-foundation-baskets-...
NL623D01Q-K11,New Look 915 Generation,"[{'sku': 'NL623D01Q-K11', 'url_key': 'new-look...","[{'key': 'discountRate', 'value': '-20%', 'tra...",False,[{'path': 'NL/62/3D/01/QK/11/NL623D01Q-K11@6.j...,PLAIN BARDOT PLAYSUIT - Combinaison - mid blue,False,False,False,False,"14,95 €","12,00 €",clothing,"[10a, 11a, 12a, 14a, 15a]",new-look-915-generation-plain-bardot-playsuit-...
GE114G01K-K11,Geox,"[{'sku': 'GE114G01K-K11', 'url_key': 'geox-kay...","[{'key': 'discountRate', 'value': '-30%', 'tra...",False,[{'path': 'GE/11/4G/01/KK/11/GE114G01K-K11@3.j...,KAYTAN BOY - Chaussures premiers pas - navy/avio,False,False,False,False,"48,95 €","34,27 €",shoe,"[20, 21, 22, 23, 24, 25]",geox-kaytan-boy-sandales-navyavio-ge114g01k-k11
JOC16D00B-Q11,Jordan,"[{'sku': 'JOC16D00B-Q11', 'url_key': 'jordan-l...","[{'key': 'discountRate', 'value': 'Jusqu’à -40...",False,[{'path': 'JO/C1/6D/00/BQ/11/JOC16D00B-Q11@9.j...,LEGACY 312 - Baskets montantes - black/varsit...,False,True,True,False,"74,95 €","45,00 €",shoe,"[27.5, 28, 28.5, 29.5, 30, 31, 31.5, 32, 33, 3...",jordan-legacy-312-baskets-montantes-blackvarsi...
TI116I010-B11,Timberland,"[{'sku': 'TI116I010-B11', 'url_key': 'timberla...","[{'key': 'discountRate', 'value': 'Jusqu’à -40...",False,[{'path': 'TI/11/6I/01/0B/11/TI116I010-B11@10....,PREMIUM BOOT - Bottines à lacets - wheat,False,True,True,False,"149,95 €","90,00 €",shoe,"[36, 37, 38, 39, 40]",timberland-premium-boot-bottines-a-lacets-whea...


#### Display the trending brand in DataFrame

In [19]:
df.brand_name.value_counts().index[0]

'Nike Sportswear'

#### Display the brand with maximal total discount (sum of discounts on all goods)

In [20]:
#Our data is still text. Convert prices into numbers:
df['price.original']=df['price.original'].str.extract('(\d*,\d*)')
df['price.promotional']=df['price.promotional'].str.extract('(\d*,\d*)')

df['price.original'] = [x.replace(',', '.') for x in df['price.original']]
df['price.promotional'] = [x.replace(',', '.') for x in df['price.promotional']]

In [21]:
df['discount_amount']=df['price.original'].astype(float)-df['price.promotional'].astype(float)
df1=df.copy()

In [22]:
total_disc=df1.groupby(['brand_name']).sum().discount_amount

In [29]:
total_disc.sort_values(ascending=False).index[0]

'Nike Sportswear'

#### Display the brands without discount at all

In [25]:
total_disc[total_disc==0]

Series([], Name: discount_amount, dtype: float64)
